<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/BCE_Repeated_Objs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BCE Vs MultiLabelSoftMarginLoss
Downloading:

In [3]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = ['1pZgspfWi1Mh8sqV_lbJLQTuFZyhnTPlA']
downloaded = drive.CreateFile({'id':id[0]}) 
downloaded.GetContentFile('localization_2.zip')
!unzip localization_2.zip

Archive:  localization_2.zip
  inflating: localization/001202.png  
  inflating: localization/001203.png  
  inflating: localization/1_frame132.png  
  inflating: localization/1_frame146.png  
  inflating: localization/16_frame119.png  
  inflating: localization/16_frame122.png  
  inflating: localization/5_frame078.png  
  inflating: localization/5_frame118.png  
  inflating: localization/combine_miccai18_ResNet50_256,320_170_best_checkpoint.pth.tar  
  inflating: localization/raw_image.png  


In [13]:
from torchvision import models
import torch.nn as nn
from torch.nn import DataParallel
import torch.nn.functional as F
from torchvision import transforms as tf
import torch
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import random
import torch
import numpy as np
import os

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 11)
model = DataParallel(model)
result_filename = 'combine_miccai18_ResNet50_256,320_170'
best_checkpoint_path = 'localization/' + result_filename + '_best_checkpoint.pth.tar'
checkpoint = torch.load(best_checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)
model = model.module
model.cuda();
model.eval()

image_name = "localization/1_frame146.png"   
classes = ['bipolar_forceps', 'prograsp_forceps', 'large_needle_driver', 'monopolar_curved_scissors', 'ultrasound_probe', 'suction', 'clip_applier', 'stapler', 'hook', 'specimen_bag', 'tissue']

'''------------------------------------- Load image ------------------------------------'''
image = Image.open(image_name).convert('RGB')
transforms = tf.Compose([tf.Resize((256,320)), 
                        tf.ToTensor(),
                        tf.Normalize([0.4084945, 0.25513682, 0.25353566], [0.22662906, 0.20201652, 0.1962526 ])])

img = transforms(image).cuda()
lab = torch.tensor([1,0,0,1,0,1,0,0,0,0,1])
img, lab = torch.stack([img, img]).cuda(), torch.stack([lab,lab])#.cuda()
logits = model(img)
print(logits.shape, lab.shape)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([2, 11]) torch.Size([2, 11])


Comparisons

In [14]:
#Version1:
import torch.nn.functional as F
prob = torch.sigmoid(logits)
loss_bce_f = F.binary_cross_entropy(prob.detach().cpu(), lab.float())
print('loss_bce_f:',loss_bce_f.item())

#Version2:
from torch.nn import BCELoss
prob = torch.sigmoid(logits)
loss_bce_nn = BCELoss()(prob.detach().cpu(), lab.float())
print('loss_bce_nn:',loss_bce_nn.item())

#Version3:
from torch.nn import BCEWithLogitsLoss
loss_bce_logits = BCEWithLogitsLoss()(logits.detach().cpu(), lab.float())
print('loss_bce_logits:',loss_bce_logits.item())

#Version4:
from torch.nn import MultiLabelSoftMarginLoss
loss_MultiLabelSoftMarginLoss = MultiLabelSoftMarginLoss()(logits.detach().cpu(), lab.float())
print('loss_MultiLabelSoftMarginLoss:',loss_MultiLabelSoftMarginLoss.item())

loss_bce_f: 0.05638723075389862
loss_bce_nn: 0.05638723075389862
loss_bce_logits: 0.05638723075389862
loss_MultiLabelSoftMarginLoss: 0.056387219578027725


# Comparison for two repeated classes:

In [27]:
image = Image.open(image_name).convert('RGB')
transforms = tf.Compose([tf.Resize((256,320)), 
                        tf.ToTensor(),
                        tf.Normalize([0.4084945, 0.25513682, 0.25353566], [0.22662906, 0.20201652, 0.1962526 ])])

img = transforms(image).cuda()

lab = [[1,0,0,1,0,1,0,0,0,0,1],
      [1,0,0,1,0,1,0,0,0,0,1]]
lab = torch.tensor(lab)
img, lab = torch.stack([img, img]).cuda(), torch.stack([lab,lab])#.cuda()

model = models.resnet50(pretrained=True)
num_classes = 11
max_repeated_objs = 2
model.fc = nn.Linear(model.fc.in_features, num_classes * max_repeated_objs)
model.cuda()
logits = model(img)
print('logit:',logits.shape, 'lab:',lab.shape)
print('lab after flatten',lab.flatten(start_dim=1).shape)

import torch.nn.functional as F
prob = torch.sigmoid(logits)
loss_bce_f = F.binary_cross_entropy(prob.detach().cpu(), lab.float().flatten(start_dim=1))
print('loss_bce_f:',loss_bce_f.item())

logit: torch.Size([2, 22]) lab: torch.Size([2, 2, 11])
lab after flatten torch.Size([2, 22])
loss_bce_f: 0.7434186935424805


In [24]:
#Version1:
import torch.nn.functional as F
prob = torch.sigmoid(logits)
loss_bce_f = F.binary_cross_entropy(prob.detach().cpu(), lab.float().flatten(start_dim=1))
print('loss_bce_f:',loss_bce_f.item())

#Version2:
from torch.nn import BCELoss
prob = torch.sigmoid(logits)
loss_bce_nn = BCELoss()(prob.detach().cpu(), lab.float().flatten(start_dim=1))
print('loss_bce_nn:',loss_bce_nn.item())

#Version3:
from torch.nn import BCEWithLogitsLoss
loss_bce_logits = BCEWithLogitsLoss()(logits.detach().cpu(), lab.float().flatten(start_dim=1))
print('loss_bce_logits:',loss_bce_logits.item())

#Version4:
from torch.nn import MultiLabelSoftMarginLoss
loss_MultiLabelSoftMarginLoss = MultiLabelSoftMarginLoss()(logits.detach().cpu(), lab.float().flatten(start_dim=1))
print('loss_MultiLabelSoftMarginLoss:',loss_MultiLabelSoftMarginLoss.item())

loss_bce_f: 0.6977637410163879
loss_bce_nn: 0.6977637410163879
loss_bce_logits: 0.6977637410163879
loss_MultiLabelSoftMarginLoss: 0.6977637410163879
